<a href="https://colab.research.google.com/github/Divyacanny22/Divyacanny22/blob/main/Face_expression_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libraries, packages and datase

In [ ]:
!git clone https://github.com/parth1620/Facial-Expression-Dataset.git
!pip install -U git+https://github.com/albumentations-team/albumentations
!pip install timm
!pip install --upgrade opencv-contrib-python

Cloning into 'Facial-Expression-Dataset'...
remote: Enumerating objects: 34052, done.
remote: Total 34052 (delta 0), reused 0 (delta 0), pack-reused 34052
Receiving objects: 100% (34052/34052), 52.31 MiB | 22.80 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (35887/35887), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/albumentations-team/albumentations to /tmp/pip-req-build-4mury6gr
  Running command git clone --filter=blob:none --quiet https://github.com/albumentations-team/albumentations /tmp/pip-req-build-4mury6gr
  Resolved https://github.com/albumentations-team/albumentations to commit b773a1aa69f9c823c7f593205614d05d32c039cb
  Preparing metadata (setup.py) ... done
  Created wheel for albumentations: filename=albumentations-1.3.0-py3-none-any.whl size=125709 sha256=ee5005d9c2d7906dc8421aa5b8a63e3ffc844d79249f0ba79e472b792a7737ab
  Stored in directory: /tmp/pip-ephem-whee

##import

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import torch 

##Configurations

In [ ]:
train_img_folder_path = '/content/Facial-Expression-Dataset/train/'
valid_img_folder_path = '//content/Facial-Expression-Dataset/validation/'

lr=0.001
batch_size=32
epochs=15

device='cuda'
model_name='efficientnet_b0'

##Load Dataset

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision import transforms as T 

In [ ]:
train_augs=T.Compose([
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=(-20,+20)),
    T.ToTensor() #PIL / numpy arr -> torch tensor -> (h,w,c) ->(c,h,w)
])

valid_augs = T.Compose([
    T.ToTensor()
])

In [ ]:
trainset=ImageFolder(train_img_folder_path, transform= train_augs)
validset=ImageFolder(valid_img_folder_path, transform=valid_augs)


In [ ]:
print(f"Total no. of example in trainset : {len(trainset)}")
print(f"Total no. of example in validset : {len(validset)}")

Total no. of example in trainset : 28821
Total no. of example in validset : 7066


In [ ]:
print(trainset.class_to_idx)

{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


In [ ]:
image, label = trainset[3]

plt.imshow(image.permute(1,2,0)) #(h,w,c)
plt.title(label);

## load dataset into batches

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_loader= DataLoader(trainset, batch_size= batch_size, shuffle=True)
valid_loader= DataLoader(validset, batch_size=batch_size)

In [ ]:
print(f"Total no. of batches in trainloader : {len(train_loader)}")
print(f"Total no. of batches in validloader : {len(valid_loader)}")

Total no. of batches in trainloader : 901
Total no. of batches in validloader : 221


In [ ]:
for images,labels in train_loader:
  break;

print(f"One image batch shape : {images.shape}")
print(f"One label batch shape : {labels.shape}")

One image batch shape : torch.Size([32, 3, 48, 48])
One label batch shape : torch.Size([32])


##Create Model

In [ ]:
import timm 
from torch import nn 

In [ ]:
class FaceModel(nn.Module):
  def __init__(self):
    super(FaceModel,self).__init__()
    self.eff_net=timm.create_model('efficientnet_b0',pretrained=True,num_classes=7)

  def forward(self, images,labels=None):
    logits=self.eff_net(images)

    if labels != None:
      loss = nn.CrossEntropyLoss()(logits, labels)
      return logits, loss

    return logits    


In [ ]:
model = FaceModel()
model.to(device)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b0_ra-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_ra-3dd342df.pth


FaceModel(
  (eff_net): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 

## Create train and Eval function

In [ ]:
from tqdm import tqdm

In [ ]:
def multiclass_accuracy(y_pred,y_true):
    top_p,top_class = y_pred.topk(1,dim = 1)
    equals = top_class == y_true.view(*top_class.shape)
    return torch.mean(equals.type(torch.FloatTensor))

In [ ]:
def train_fn(model, dataloader,optimizer, current_epo):

  model.train()
  total_loss=0.0
  total_acc=0.0
  tk=tqdm(dataloader, desc="EPOCH"+"[TRAIN]"+str(current_epo+1)+"/"+str(epochs))

  for t,data in enumerate(tk):
    images,labels=data
    images,labels = images.to (device), labels.to(device)

    optimizer.zero_grad()
    logits, loss = model(images, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    total_acc += multiclass_accuracy(logits, labels)
    tk.set_postfix({'lose' : '%6f' %float(total_loss / (t+1)), 'acc' : '%6f' %float(total_acc / (t+1)),})
  return total_loss / len(dataloader), total_acc / len(dataloader)  

In [ ]:
def eval_fn(model, dataloader, current_epo):

  model.eval()
  total_loss=0.0
  total_acc=0.0
  tk=tqdm(dataloader, desc="EPOCH"+"[VALID]"+str(current_epo+1)+"/"+str(epochs))

  for t,data in enumerate(tk):
    images,labels=data
    images,labels = images.to (device), labels.to(device)

    
    logits, loss = model(images, labels)
 

    total_loss += loss.item()
    total_acc += multiclass_accuracy(logits, labels)
    tk.set_postfix({'lose' : '%6f' %float(total_loss / (t+1)), 'acc' : '%6f' %float(total_acc / (t+1)),})
  return total_loss / len(dataloader), total_acc / len(dataloader)  

## Create training loop

In [ ]:
optimizer= torch.optim.Adam(model.parameters(),lr=lr)

In [ ]:
best_valid_loss =np.Inf

for i in range(epochs):
  train_loss, train_acc=train_fn(model,train_loader,optimizer,i)
  valid_loss, valid_acc=eval_fn(model,valid_loader,i)

  if valid_loss < best_valid_loss:
    torch.save(model.state_dict(), 'best_weights.pt')
    print("SAVED BEST-WEIGHTS")
    best_valid_loss=valid_loss


EPOCH[VALID]1/15: 100%|██████████| 221/221 [00:06<00:00, 35.21it/s, lose=1.193276, acc=0.543585]


SAVED BEST-WEIGHTS


EPOCH[VALID]2/15: 100%|██████████| 221/221 [00:06<00:00, 36.45it/s, lose=1.117023, acc=0.575563]


SAVED BEST-WEIGHTS


EPOCH[VALID]3/15: 100%|██████████| 221/221 [00:06<00:00, 34.67it/s, lose=1.104526, acc=0.585037]


...

EPOCH[VALID]5/15: 100%|██████████| 221/221 [00:06<00:00, 36.39it/s, lose=1.039591, acc=0.613383]


SAVED BEST-WEIGHTS


EPOCH[VALID]6/15: 100%|██████████| 221/221 [00:06<00:00, 35.45it/s, lose=1.014374, acc=0.620986]


SAVED BEST-WEIGHTS


EPOCH[TRAIN]7/15:  69%|██████▉   | 625/901 [00:46<00:19, 14.32it/s, lose=0.973425, acc=0.637709]

In [ ]:
def view_classify(img, ps):
    
    classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

    ps = ps.data.cpu().numpy().squeeze()
    img = img.numpy().transpose(1,2,0)
   
    fig, (ax1, ax2) = plt.subplots(figsize=(5,9), ncols=2)
    ax1.imshow(img)
    ax1.axis('off')
    ax2.barh(classes, ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(classes)
    ax2.set_yticklabels(classes)
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()

    return None